# Python Abuse

### August 2021

### Lewis Gaul

In [ ]:
# SETUP

import inspect
import logging
import yaml
from pprint import pprint

def send_email(to, msg):
    logging.warning(f"Sending email to {to!r}...")

## Operator overloading

It's possible to overload any of:
  +, -, *, @, /, //, %, **, <<, >>, &, ^, |, >, <, ==

or any of these followed by '=' (except '=='). Can also overload keyword behaviour, such as 'in'.

In [ ]:
class Functional:
    def __init__(self):
        self.funcs = []
    def __matmul__(self, other):
        self.funcs.clear()
        self.funcs.append(other)
        return self
    def __rshift__(self, other):
        self.funcs.append(other)
        return self
    def __le__(self, other):
        result = other
        for f in self.funcs:
            result = f(result)
        return result
    
_ = Functional()

In [ ]:
square = lambda x: x * x
make_add = lambda n: (lambda x: x + n)
add1 = make_add(1)

result = _@ add1 >> square >> make_add(10)  <= -2
print(f"{result = }")

mycalc = _@ (lambda x: 2*x + 1) >> square
print(f"{(mycalc <= 1) = }")

## Decorators

Decorators run at function *definition* time - may have side effects even if the decorated function isn't called.

They also overwrite the object stored in the function's name (not necessarily a function!).

In [ ]:
def decorator(func):
    global dr
    import data_reader as dr
    send_email(to="ensoft-all@cisco.com", msg="Decorator ran :)")
    with open(func.__name__ + ".yaml") as f:
        return dr.parse_data(schema=func(), node=yaml.safe_load(f))

In [ ]:
@decorator
def int_val():
    return dr.Int

In [ ]:
!cat int_val.yaml

In [ ]:
print(int_val)
print(mycalc <= int_val)

## Metaclasses

Define behaviour of classes... Probably the most powerful tool in the Python toolbox :)

In [ ]:
class MyMeta(type):
    def __new__(mcs, name, bases, attrs):
        for name, value in attrs.items():
            if inspect.isfunction(value):
                attrs[name] = decorator(value)
        return super().__new__(mcs, name, bases, attrs)

In [ ]:
class MyClass(metaclass=MyMeta):
    def my_data():
        return dr.List(
            dr.UserClass["Entry"](
                name=dr.Str.restrict(max_len=20),
                link=dr.Str.restrict(regex="https?://.+") | None,
                runs_on=dr.List(
                    dr.UserEnum["RunsOn"]("server", "local", "web")
                ),
                maintained=dr.Bool,
            ).defaults(link=None, runs_on=list, maintained=True)
        )

In [ ]:
!cat my_data.yaml

In [ ]:
pprint(MyClass.my_data)